<a href="https://colab.research.google.com/github/XDMickeyYau/CSC2515-Project/blob/simCLR-results/simCLR/feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib
import matplotlib.pyplot as plt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r drive/MyDrive/CIFARData/*.npz .

In [5]:
CIFARData = np.load('CIFARData10Class.npz')['arr_0']
CIFARDataTest = np.load('CIFARDataTest10Class.npz')['arr_0']
CIFARDataVal = np.load('CIFARValData10Class.npz')['arr_0']
CIFARData.shape, CIFARDataTest.shape, CIFARDataVal.shape

((10, 4500, 32, 32, 3), (10, 1000, 32, 32, 3), (10, 500, 32, 32, 3))

In [6]:
CIFARDataFlat = CIFARData.reshape(-1,32,32,3)
CIFARDataTestFlat = CIFARDataTest.reshape(-1,32,32,3)
CIFARDataValFlat = CIFARDataVal.reshape(-1,32,32,3)
CIFARDataFlat.shape, CIFARDataTestFlat.shape, CIFARDataValFlat.shape

((45000, 32, 32, 3), (10000, 32, 32, 3), (5000, 32, 32, 3))

In [7]:
hub_path = 'gs://simclr-checkpoints/simclrv2/finetuned_100pct/r50_1x_sk0/hub/'
module = hub.Module(hub_path, trainable=False)

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [9]:
def convert_feature(module, sess, data):
    input = tf.convert_to_tensor(data.astype(np.float32))
    feature_vec = sess.run(module(input))
    return feature_vec

In [10]:
CIFARDataFeature = convert_feature(module, sess, CIFARDataFlat)
CIFARDataTestFeature = convert_feature(module, sess, CIFARDataTestFlat)
CIFARDataValFeature = convert_feature(module, sess, CIFARDataValFlat)
CIFARDataFeature.shape, CIFARDataTestFeature.shape, CIFARDataValFeature.shape

((45000, 2048), (10000, 2048), (5000, 2048))

In [11]:
np.savez('CIFARDataFeature10Class.npz', CIFARDataFeature)
np.savez('CIFARDataTestFeature10Class.npz', CIFARDataTestFeature)
np.savez('CIFARDataValFeature10Class.npz', CIFARDataValFeature)

In [13]:
!cp -r ./*.npz drive/MyDrive/simCLRfeature/